Copyright (c) 2023, Douglas Santry
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, is permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [1]:
import numpy as np

In [2]:
def DisplayBoard (T):
    print ("Board:\n")
    for i in range (0, 3):
        for j in range (0, 3):
            if T[i][j] == 0:
                print ("_", end="")
            elif T[i][j] == 1:
                print ("X", end="")
            else:
                print ("O", end="")

        print ("\n")

In [3]:
def TicTacToeTest (T, condition):

    down = 0
    up = 0
    
    for row in range (0, 3):
        
        score = 0

        for column in range (0, 3):
            score = score + T[row][column]

        if (abs (score) == 3):
            return (True if score == condition else False)
        
        down = down + T[row][row]
        up = up + T[row][2 - row]

    if (abs (down) == 3):
            return (True if down == condition else False)
    if (abs (up) == 3):
            return (True if up == condition else False)

    for column in range (0, 3):
        
        score = 0

        for row in range (0, 3):
            score = score + T[row][column]

        if (abs (score) == 3):
            return (True if score == condition else False)

    return False

ZeroSumGame = TicTacToeTest

In [4]:
#
# Set up global stuff
#

T0 = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
T_Empty = np.copy (T0)

# a0 = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
a0 = [0, 0, 0, 0, 0, 0, 0, 0, 0]

DPState = { str (T0): a0 }       # actions
DPCount = { str (T0): 0 }        # k, for the updates

epsilon = 0.5

moves = 0
explored = 0

In [5]:
def TupleFromLinear (index):
    row = int (index / 3)
    column = int (index % 3)
    
    return row, column

In [6]:
def SelectAction (T):

    global DPState
    global DPCount
    global epsilon
    global explored

    actions = DPState.get (str (T))
    if actions == None:
        actions = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
        DPState[str (T)] = actions
        DPCount[str (T)] = 0

    explore = np.random.uniform (0, 1, 1)
    if explore[0] > epsilon:
        explored += 1
        return TupleFromLinear (int (np.random.uniform (0, 9, 1)[0]))

    action = actions.index (max (actions))
    
    return TupleFromLinear (action)


In [7]:
def UpdateAction (T, row, column, update, useRule=False):
    
    index = 3 * row + column
    key = str (T)

    actions = DPState.get (key)
    k = DPCount.get (key)

    # None is "impossible" here
    if useRule and k > 0:
        reward = actions[index]
        reward = reward + 1/k * (update - reward)
        actions[index] = reward
    else:
        actions[index] = update

    k += 1
    DPCount[key] = k

In [8]:
def Opponent (depth, T, player):

    global moves
    moves += 1

    available = 9 - depth + 1
    attempts = 0

    actions = list (np.random.uniform (0, 1, available))
    action = actions.index (max (actions)) + 1 # Not an index, but number of attempts

    for row in range (0, 3):
        for column in range (0, 3):

            if T[row][column] == 0:
                attempts += 1
            
            if attempts == action:
                T[row][column] = player
                won = ZeroSumGame (T, -3)
                if won:
                    T[row][column] = 0
                    return 0.0
                elif depth < 9:
                    reward = Traverse (depth + 1, T, -player)
                    T[row][column] = 0
                    return reward
                else:
                    T[row][column] = 0
                    return 0

            if attempts > 9:
                print ("IMPOSSIBLE", depth, T)
                return
        
    return 0.0 # treating a draw like a loss


In [9]:
def Traverse (depth, T, player):

    global moves
    moves += 1

    while True:
        
        row, column = SelectAction (T)
        reward = 0.0

        if T0[row][column] == 0:
                
            T[row][column] = player
            won = ZeroSumGame (T, 3)
            if won:
                T[row][column] = 0
                UpdateAction (T, row, column, 1.0)
                return 1.0
                
            if depth < 9:
                reward = Opponent (depth + 1, T, -player)
                T[row][column] = 0
                UpdateAction (T, row, column, reward, True)
            else:
                T[row][column] = 0
                UpdateAction (T, row, column, 0.0)
                
            return reward
        else:
            UpdateAction (T, row, column, 0.0) # action unavailable in this state


In [10]:
for i in range (0, 100000):
    Traverse (1, T0, 1)


In [11]:
moves, explored

(635944, 254288)

In [12]:
# DPState

In [13]:
len (DPCount)

2423

In [14]:
def OpponentPlaying (depth, T, player):

    available = 9 - depth + 1
    attempts = 0

    actions = list (np.random.uniform (0, 1, available))
    action = actions.index (max (actions)) + 1 # Not an index, but number of attempts

    for row in range (0, 3):
        for column in range (0, 3):

            if T[row][column] == 0:
                attempts += 1
            
            if attempts == action:
                T[row][column] = player
                won = ZeroSumGame (T, -3)
                if won:
                    return -1
                elif depth < 9:
                    return Agent (depth + 1, T, -player)
                else:
                    return 0

            if attempts > 9:
                print ("IMPOSSIBLE", depth, T)
                return

    return 0.0 # treating a draw like a loss


In [15]:
def Agent (depth, T, player):

    global moves
    moves += 1

    while True:
        
        row, column = SelectAction (T)
        reward = 0.0

        if T0[row][column] == 0:
                
            T[row][column] = player
            won = ZeroSumGame (T, 3)
            if won:
                return 1
                
            elif depth < 9:
                return OpponentPlaying (depth + 1, T, -player)

            return 0


In [16]:
T0 = T_Empty.copy ()

In [17]:
T0 = T_Empty.copy ()
Agent (1, T0, 1)


1

In [18]:
T0

array([[ 1,  0,  0],
       [-1,  1,  0],
       [-1,  0,  1]])

metric = 0
epsilon = 0.0

for i in range (0, 10):
    T0 = T_Empty.copy ()
    success = Agent (1, T0, 1)
    if success == 1:
        metric += 1
        print ("WON!")
    elif success == -1:
        print (":-(")
    else:
        print ("DRAW")

    DisplayBoard (T0)

metric

In [19]:
wins = 0
losses = 0
draws = 0
trials = 10000

for i in range (0, trials):
    T0 = T_Empty.copy ()
    success = Agent (1, T0, 1)
    if success == 1:
        wins += 1
    elif success == -1:
        losses += 1
    else:
        draws += 1

    # DisplayBoard (T0)
print ("Wins\tDraws\tLosses")
wins/trials, losses/trials, draws/trials

Wins	Draws	Losses


(0.6041, 0.2643, 0.1316)

In [20]:
DPState


{'[[0, 0, 0], [0, 0, 0], [0, 0, 0]]': [0.3354732737213709,
  0.28018784756187776,
  0.3944598281086567,
  0.24716125930614963,
  0.899171317084295,
  0.26731644111470354,
  0.29818607001279845,
  0.246454787734499,
  0.3277031360763447],
 '[[1, 0, 0], [0, 0, 0], [0, 0, -1]]': [0.0,
  0.381953813233393,
  0.5623656740088826,
  0.537513376751374,
  0.45790430149679906,
  0.7179436487244587,
  0.12325939534924242,
  0.5277336948693475,
  0.0],
 '[[1, 0, 0], [-1, 0, 0], [1, 0, -1]]': [0.0,
  0.34233308180539485,
  0.8737392741536706,
  0.0,
  0.5238579295782685,
  0.4895833333333333,
  0.0,
  0.47919217931395924,
  0.0],
 '[[1, 1, 0], [-1, 0, -1], [1, 0, -1]]': [0.0,
  0.0,
  1.0,
  0.0,
  0.5,
  0.5,
  0.5,
  0.0,
  0.5],
 '[[0, 0, 0], [0, 1, 0], [-1, 0, 0]]': [0.9138945351310602,
  0.6455991231617947,
  0.5169187425698837,
  0.5967969450304372,
  0.0,
  0.2740131438654699,
  0.0,
  0.6482085416311125,
  0.6425574973789252],
 '[[1, 0, 0], [0, 1, -1], [-1, 0, 0]]': [0.0,
  0.98408608251923